In [1]:
from math import ceil
import tensorflow as tf
import tensorflow_model_optimization.sparsity.keras as sparsity

from utils.metrics import count_nonzero_params
from prune_model import prune_layer

In [2]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [3]:
batch_size = 64

model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(32, 32, 3)),
    tf.keras.layers.DepthwiseConv2D(kernel_size=4),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation="softmax")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[
        tf.keras.metrics.TopKCategoricalAccuracy(k=1, name="top1"),
        tf.keras.metrics.TopKCategoricalAccuracy(k=3, name="top3")
    ]
)

model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=batch_size,
    epochs=5
)

Epoch 1/5
782/782 [==============================] - 21s 27ms/step - loss: 29.4837 - top1: 0.1854 - top3: 0.4601 - val_loss: 6.5115 - val_top1: 0.1991 - val_top3: 0.4741
Epoch 2/5
782/782 [==============================] - 21s 26ms/step - loss: 5.3428 - top1: 0.2247 - top3: 0.5102 - val_loss: 3.9002 - val_top1: 0.2211 - val_top3: 0.5031
Epoch 3/5
782/782 [==============================] - 22s 28ms/step - loss: 3.5198 - top1: 0.2441 - top3: 0.5263 - val_loss: 3.2264 - val_top1: 0.2400 - val_top3: 0.5234
Epoch 4/5
782/782 [==============================] - 21s 27ms/step - loss: 2.9604 - top1: 0.2524 - top3: 0.5393 - val_loss: 2.9437 - val_top1: 0.2196 - val_top3: 0.4957
Epoch 5/5
782/782 [==============================] - 21s 26ms/step - loss: 2.6982 - top1: 0.2600 - top3: 0.5480 - val_loss: 2.6575 - val_top1: 0.2543 - val_top3: 0.5309


In [4]:
initial_sparsity = 0
final_sparsity = 0.9
start_pruning = 0
pruning_steps = 5
frequency = 2

epochs = start_pruning + pruning_steps * frequency
steps_per_epoch = ceil(len(X_train) // batch_size + 1)

pruning_schedule = sparsity.PolynomialDecay(
    initial_sparsity=initial_sparsity,
    final_sparsity=final_sparsity,
    begin_step=start_pruning*steps_per_epoch,
    end_step=epochs*steps_per_epoch,
    frequency=frequency*steps_per_epoch
)

pruning_model = tf.keras.models.clone_model(
    model=model, 
    clone_function=lambda layer: prune_layer(layer, pruning_schedule)
)

/home/juho/Documents/Uni/Semester_6/Bachelorarbeit/Embedded-Systems-Bachelorarbeit/Implementierung/.venv/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer.py:2281: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


In [5]:
for l1, l2 in zip(pruning_model.layers, model.layers):
    print(type(l1).__name__, "---", type(l2).__name__)

PruneLowMagnitude --- DepthwiseConv2D
PruneLowMagnitude --- Flatten
PruneLowMagnitude --- Dense


In [6]:
pruning_model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[
        tf.keras.metrics.TopKCategoricalAccuracy(k=1, name="top1"),
        tf.keras.metrics.TopKCategoricalAccuracy(k=3, name="top3")
    ]
)

pruning_model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=batch_size,
    epochs=epochs,
    callbacks=[
        sparsity.UpdatePruningStep()
    ]
)

pruning_model = sparsity.strip_pruning(pruning_model)

Epoch 1/10
782/782 [==============================] - 22s 27ms/step - loss: 3.0933 - top1: 0.2331 - top3: 0.5179 - val_loss: 2.8012 - val_top1: 0.2383 - val_top3: 0.5183
Epoch 2/10
782/782 [==============================] - 21s 26ms/step - loss: 2.6464 - top1: 0.2578 - top3: 0.5503 - val_loss: 2.6347 - val_top1: 0.2622 - val_top3: 0.5421
Epoch 3/10
782/782 [==============================] - 21s 26ms/step - loss: 2.2361 - top1: 0.2912 - top3: 0.5926 - val_loss: 2.2995 - val_top1: 0.2692 - val_top3: 0.5629
Epoch 4/10
782/782 [==============================] - 21s 26ms/step - loss: 2.1027 - top1: 0.3056 - top3: 0.6095 - val_loss: 2.1635 - val_top1: 0.2737 - val_top3: 0.5766
Epoch 5/10
782/782 [==============================] - 21s 27ms/step - loss: 2.0981 - top1: 0.3048 - top3: 0.6082 - val_loss: 2.0884 - val_top1: 0.2981 - val_top3: 0.5963
Epoch 6/10
782/782 [==============================] - 21s 26ms/step - loss: 1.9471 - top1: 0.3297 - top3: 0.6371 - val_loss: 2.2627 - val_top1: 0.2364

In [7]:
for layer in pruning_model.layers:
    sparsity = (1 - count_nonzero_params(layer) / layer.count_params()) if layer.count_params() > 0 else 0.0
    print("[{: >5.2f}%] {}".format(sparsity * 100, type(layer).__name__))

[ 0.00%] DepthwiseConv2D
[ 0.00%] Flatten
[89.24%] Dense
